In [1]:
import sys
import os
sys.path.insert(0, '../icenumerics')
import icenumerics as ice
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm.notebook as tqdm

ureg = ice.ureg

from importlib import reload  
import From_Cairo_to_Shakti as lattices
reload(lattices)

from icenumerics.geometry import ordering 

%load_ext autoreload
%autoreload 2


import csv as csv
#import tqdm.auto as tqdm
from multiprocessing import Pool

In [2]:
output = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-16.0"

# From Cairo to Shakti lattice Simulations

In this script, I will run simulations with open boundaries. I will perform 10 experiments for each theta value.

In [3]:
def do_everything(exp_entry):
    
    e = exp_entry[1].e 
    theta = exp_entry[1].theta
    
    d = 13 # Interparticle distance
    trap_sep_l = 10 # trap_sep parameter of the edge connecting 3 coordination and 4 coordination vertices
    trap_sep_a = 4.5258
    s_l4 = d*np.sqrt(2)/2

    # <Generating lattice parameters depending on theta value>
    trap_sep_a_var, s_l3, a, l, plaquette_cte = lattices.lattice_parameters(theta, d, trap_sep_l, s_l4)
    
    # <Generating the spin lattice>
    sp = lattices.spins()
    ureg = ice.ureg
    Sx = 10 # size of the system
    Sy = 10 # size of the system
    sp.create_lattice(geometry = "from_cairo_to_shakti", size =[Sx,Sy], theta = theta,d = d ,trap_sep_l = trap_sep_l ,s_l4 = s_l4, plaquette_cte = plaquette_cte*ureg.um, border="periodic")
    sp.order_spins(ordering.random_ordering)
    
    # <Defining traps and colloids>
    particle = ice.particle(radius = 5*ureg.um,
             susceptibility = 0.005,
             diffusion = 0.125*ureg.um**2/ureg.s,
             temperature = 300*ureg.K,
             density = 1000*ureg.kg/ureg.m**3)

    trap_long = ice.trap(trap_sep = trap_sep_l*ureg.um,
                   height = 25*ureg.pN*ureg.nm,
                   stiffness = 0.1*ureg.fN/ureg.nm)

    trap_short = ice.trap(trap_sep = trap_sep_a*ureg.um,
                   height = 25*ureg.pN*ureg.nm,
                   stiffness = 0.1*ureg.fN/ureg.nm)

    traps = []
    
    for s in sp:
    
        if ( s.direction[0].magnitude == 2*trap_sep_l or s.direction[0].magnitude == -2*trap_sep_l or 
            s.direction[1].magnitude == 2*trap_sep_l or s.direction[1].magnitude == -2*trap_sep_l ):

            traps.append(trap_short)

        else:

            traps.append(trap_long)
            
    col = ice.colloidal_ice(sp, particle, traps, height_spread = 0, susceptibility_spread = 0.1)
    col.pad_region(10*ureg.um) 
    
    # <Defining simulation parameters>
    world = ice.world(
        field = 25*ureg.mT,
        temperature = 300*ureg.K,
        dipole_cutoff = 200*ureg.um)
    
    change_m_time = 2000*ureg.s
    total_time = 2500*ureg.s
    col.simulate(world,
                 name = f"ACI_from_Cairo_to_Shakti_Lattice_25mT-2500s_FreeBound_theta{theta}_exp{e}_keepingTLength",
                 include_timestamp = False,
                 targetdir = output,
                 framerate = 1*ureg.Hz,
                 timestep = 10*ureg.ms,
                 run_time = total_time,
                 output = ["x","y","z","mux","muy","muz"])
    
    col.sim.field.fieldz = "(time<2000000000)*v_Bmag*time/%f+(time>=2000000000)*v_Bmag"%(change_m_time.to(ureg.us).magnitude)
    col.run_simulation()
    col.load_simulation(slice(0,None,1))

    # <Generating the vertex array>
    four_coord, three_coord = lattices.from_Cairo_to_Shakti_vertices(Sx,Sy, theta, a, l, plaquette_cte)
    vertices_pos = np.append(four_coord,three_coord, axis = 0)
    # <Find traps_position>
    trap_pos = lattices.extracting_trap_positions(col)
    # <Infer edges dataframe>
    df = lattices.infer_edges(vertices_pos,trap_pos)
    
    v_non_inferred = ice.vertices(
        positions = vertices_pos,
        edges = df[["start","stop"]].values)
    
    # <Extract vertices dataframe and save it>
    frames = col.trj.index.get_level_values("frame").unique()

    v_df = []

    for f in tqdm.tqdm(frames[::10]):
        col.set_state_from_frame(f)

        v_non_inferred.update_directions(col)
        v_non_inferred.classify_vertices(ice.ice_to_spins(col))
        vertices = v_non_inferred.vertices

        v_df.append(vertices.copy())


    v_df = pd.concat(v_df,keys=frames[::10], names = ["frame"])
    
    v_df.to_csv(os.path.join(output,f"ACI_from_Cairo_to_Shakti_Lattice_25mT-2500s_FreeBound_theta{theta}_exp{e}_keepingTLength.dat"), sep='\t')

In [4]:
e =  np.arange(0,20)
theta = list(np.linspace(np.pi/6, 5*np.pi/180, num= 15, endpoint=True, retstep=False, dtype=None, axis=0))
L, E = np.meshgrid(theta,e)
experiments = pd.DataFrame({"e":E.flatten(),"theta":L.flatten()})
experiments

,e,theta
0,0,0.523599
1,0,0.492432
2,0,0.461266
3,0,0.430099
4,0,0.398932
...,...,...
295,19,0.211933
296,19,0.180766
297,19,0.149600
298,19,0.118433


In [ ]:
# %%time
if __name__ ==  '__main__': 
    num_processors = 5
    p=Pool(processes = num_processors)
    
    ## Create index text file
    if not os.path.exists(output):
        os.makedirs(output)
    with open(os.path.join(output,"index_ACI_from_Cairo_to_Shakti_Lattice_25mT-2500s_FreeBound_keepingTLength.dat"),'w',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow(["filename", "theta", "exp"])
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)))

/home/carolina/CairoLattice_ACI_PROJECT/Scripts/From_Cairo_to_Shakti.py:227: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/From_Cairo_to_Shakti.py:227: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY


/home/carolina/CairoLattice_ACI_PROJECT/Scripts/From_Cairo_to_Shakti.py:227: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/From_Cairo_to_Shakti.py:227: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/From_Cairo_to_Shakti.py:227: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY


scanning file


scanning file


scanning file


...

scanning file


scanning file


scanning file


scanning file


scanning file
